In [1]:
import pandas as pd
import numpy as np
import datetime
import paramiko
import os

print(os.getcwd())

/home/jian/Projects/Big_Lots/Analysis/2020_Q1/bigheroes_segments_matching


In [2]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "vgn5UucsUNHL4n9R" #hard-coded
username = "biglots_data" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [3]:
list_files_need=sftp.listdir("./")
list_files_need=[x for x in list_files_need if "bighero" in x]
list_files_need

['bigheroes_thru0502.txt', 'bigheroes_post0502.txt']

In [4]:
for file in list_files_need:
    sftp.get(file,"./"+file)
sftp.close()
transport.close()

In [5]:
df_1_thru0502=pd.read_table("./bigheroes_thru0502.txt")
print(df_1_thru0502.shape)
df_1_thru0502=df_1_thru0502.drop_duplicates()
print(df_1_thru0502.shape)


(315215, 1)
(315215, 1)


In [6]:
df_2_post0502=pd.read_table("./bigheroes_post0502.txt")
print(df_2_post0502.shape)
df_2_post0502=df_2_post0502.drop_duplicates()
print(df_2_post0502.shape)

(381734, 1)
(381734, 1)


In [11]:
df_q1_segments=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2020_Q1/BL_2020Q1_all_segments_aggregated_duplicated_JL_2020-05-13.csv",
                           usecols=['customer_id_hashed','segment'])
# remove the rows that the values str starts with "C"
df_q1_segments=df_q1_segments[df_q1_segments['segment'].apply(lambda x: x[:2]!="C_")]
print(df_q1_segments.shape,df_q1_segments['customer_id_hashed'].nunique())

(19400952, 2) 19400952


In [12]:
list_q1_seg=df_q1_segments['segment'].unique().tolist()
df_q1_seg=pd.DataFrame({"segment":list_q1_seg})
df_q1_seg['status']=df_q1_seg['segment'].apply(lambda x: x.split("_")[2])
df_q1_seg['status']=df_q1_seg['status'].replace("RFM20Q1","others_aggregated_small")
df_q1_seg['status']=df_q1_seg['status'].replace("P","new_signed_in_Q1").replace("S","new_signed_in_Q1").replace("T","new_signed_in_Q1")
df_q1_seg.shape

(293, 2)

In [13]:
df_q1_segments=pd.merge(df_q1_segments,df_q1_seg,on="segment",how="left")
df_q1_segments_summary=df_q1_segments.groupby("status")['customer_id_hashed'].count().to_frame().reset_index()


In [14]:
df_1_thru0502=pd.merge(df_1_thru0502,df_q1_segments,left_on="hashed_customer_num",right_on="customer_id_hashed",how="left")
df_1_thru0502['status']=df_1_thru0502['status'].fillna("balanced")
df_1_thru0502.head(3)

,hashed_customer_num,customer_id_hashed,segment,status
0,24eb3ed5db1c6649e3a866e5fb9664e8f9811976c2169a...,24eb3ed5db1c6649e3a866e5fb9664e8f9811976c2169a...,T_Legacy_Active_M_S_Above50K_SingleFamily_Comp...,Active
1,2f92e2811f2b15059d80de0dded6735fedb55a1fa4c001...,NaN,NaN,balanced
2,007d845f4549a34938f2ce8a26ccb1b9e737d65dda6213...,007d845f4549a34938f2ce8a26ccb1b9e737d65dda6213...,T_Legacy_Active_M_P_Above50K_SingleFamily_Comp...,Active


In [15]:
df_1_thru0502_summary=df_1_thru0502.groupby("status")['hashed_customer_num'].count().to_frame().reset_index()

In [17]:
import glob

list_q2_segs=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q2/upload_files/*.csv")
list_q2_segs=[x for x in list_q2_segs if "renamed" not in x]
list_q2_segs=[x for x in list_q2_segs if os.path.basename(x)[0].lower()!="c"]

len(list_q2_segs)

80

In [18]:
df_q2_segments=pd.DataFrame()
list_cols=[]
for file in list_q2_segs:
    df=pd.read_csv(file,usecols=['customer_id_hashed','segment_2020Q2'])
    df_q2_segments=df_q2_segments.append(df)
df_q2_segments.shape,df_q2_segments['customer_id_hashed'].nunique()

((17493297, 2), 17493297)

In [19]:
list_q2_seg=df_q2_segments['segment_2020Q2'].unique().tolist()
df_q2_seg=pd.DataFrame({"segment_2020Q2":list_q2_seg})
df_q2_seg['status']=df_q2_seg['segment_2020Q2'].apply(lambda x: x.split("_")[2])
df_q2_seg['status']=df_q2_seg['status'].replace("RFM20Q2","others_aggregated_small")
# to append new sign_ups 

In [20]:
df_q2_segments=pd.merge(df_q2_segments,df_q2_seg,on="segment_2020Q2",how="left")

In [21]:
def recursive_files(root_p):
    for root, dirs, files in os.walk(root_p):
        for file in files:
            yield os.path.join(root,file)

In [22]:
list_files_new_sign_ups_Q2=list(recursive_files("/home/jian/BigLots/2020_by_weeks/"))
list_files_new_sign_ups_Q2=[x for x in list_files_new_sign_ups_Q2 if "master" in x.lower()]
list_files_new_sign_ups_Q2=[x for x in list_files_new_sign_ups_Q2 if x.split("Storm_")[1][:10]>"2020-05-02"]
df_new_sing_ups=pd.DataFrame()

for file in list_files_new_sign_ups_Q2:
    df=pd.read_csv(file,usecols=['customer_id_hashed'],sep="|")
    df_new_sing_ups=df_new_sing_ups.append(df)
df_new_sing_ups['status']='new_signed_in_Q2'

df_q2_segments=df_q2_segments.append(df_new_sing_ups)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [23]:
df_q2_segments_summary=df_q2_segments.groupby("status")['customer_id_hashed'].count().to_frame().reset_index()

In [24]:
df_2_post0502=pd.merge(df_2_post0502,df_q2_segments,left_on="hashed_customer_num",right_on="customer_id_hashed",how="left")
df_2_post0502['status']=df_2_post0502['status'].fillna("balanced")
df_2_post0502_summary=df_2_post0502.groupby("status")['hashed_customer_num'].count().to_frame().reset_index()

In [26]:
writer=pd.ExcelWriter("./BL_bigheroes_matched_with_target_IDs_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_q1_segments_summary.to_excel(writer,"q1_segs",index=False)
df_1_thru0502_summary.to_excel(writer,"thru0502_summary",index=False)

df_q2_segments_summary.to_excel(writer,"q2_segs",index=False)
df_2_post0502_summary.to_excel(writer,"post0502_summary",index=False)
writer.save()